In [3]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\laksh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
amazon_df = pd.read_csv("amazon_product.csv")
amazon_df

,id,Title,Description,Category
0,1,Swissmar Capstore Select Storage Rack for 18-...,Swissmar's capstore select 18 storage unit kee...,Home & Kitchen Kitchen & Dining Kitchen Utens...
1,2,Gemini200 Delta CV-880 Gold Crown Livery Airc...,Welcome to the exciting world of GeminiJets! O...,Toys & Games Hobbies Models & Model Kits Pre-...
2,5,Superior Threads 10501-2172 Magnifico Cream P...,"For quilting and embroidery, this product is m...","Arts, Crafts & Sewing Sewing Thread & Floss S..."
3,6,Fashion Angels Color Rox Hair Chox Kit,Experiment with the haute trend of hair chalki...,Beauty & Personal Care Hair Care Hair Colorin...
4,8,Union Creative Giant Killing Figure 05: Daisu...,From Union Creative. Turn your display shelf i...,Toys & Games › Action Figures & Statues › Sta...
...,...,...,...,...
663,2482,Rosemery (Rosemary) - Box of Six 20 Stick Hex...,"Six tubes, each containing 20 sticks of incens...",Home & Kitchen Home Décor Home Fragrance Ince...
664,2485,"InterDesign Linus Stacking Organizer Bin, Ext...",The InterDesign Linus Organizer Bins are stack...,Home & Kitchen Kitchen & Dining Storage & Org...
665,2489,Gourmet Rubber Stamps Diagonal Stripes Stenci...,Gourmet Rubber Stamps-Stencil. This delicious ...,Toys & Games Arts & Crafts Printing & Stamping
666,2496,Spenco RX Arch Cushion Full Length Comfort Su...,"Soft, durable arch support. consumers with gen...",Health & Household › Health Care › Foot Healt...


In [5]:
amazon_df.shape

(668, 4)

In [6]:
amazon_df.drop("id",axis=1,inplace=True)

In [7]:
amazon_df.head()

,Title,Description,Category
0,Swissmar Capstore Select Storage Rack for 18-...,Swissmar's capstore select 18 storage unit kee...,Home & Kitchen Kitchen & Dining Kitchen Utens...
1,Gemini200 Delta CV-880 Gold Crown Livery Airc...,Welcome to the exciting world of GeminiJets! O...,Toys & Games Hobbies Models & Model Kits Pre-...
2,Superior Threads 10501-2172 Magnifico Cream P...,"For quilting and embroidery, this product is m...","Arts, Crafts & Sewing Sewing Thread & Floss S..."
3,Fashion Angels Color Rox Hair Chox Kit,Experiment with the haute trend of hair chalki...,Beauty & Personal Care Hair Care Hair Colorin...
4,Union Creative Giant Killing Figure 05: Daisu...,From Union Creative. Turn your display shelf i...,Toys & Games › Action Figures & Statues › Sta...


In [8]:
amazon_df.isnull().sum()

Title          0
Description    0
Category       0
dtype: int64

In [9]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [10]:
def tokenize_stem(text):
  tokens = nltk.word_tokenize(text.lower())
  stemmed = [stemmer.stem(word) for word in tokens]
  return " ".join(stemmed)

In [11]:
amazon_df["stemmed_tokens"] = amazon_df.apply(lambda row: tokenize_stem(row["Title"] + " " + row["Description"]),axis=1)

In [12]:
amazon_df.head()

,Title,Description,Category,stemmed_tokens
0,Swissmar Capstore Select Storage Rack for 18-...,Swissmar's capstore select 18 storage unit kee...,Home & Kitchen Kitchen & Dining Kitchen Utens...,swissmar capstor select storag rack for 18-pac...
1,Gemini200 Delta CV-880 Gold Crown Livery Airc...,Welcome to the exciting world of GeminiJets! O...,Toys & Games Hobbies Models & Model Kits Pre-...,gemini200 delta cv-880 gold crown liveri aircr...
2,Superior Threads 10501-2172 Magnifico Cream P...,"For quilting and embroidery, this product is m...","Arts, Crafts & Sewing Sewing Thread & Floss S...",superior thread 10501-2172 magnifico cream puf...
3,Fashion Angels Color Rox Hair Chox Kit,Experiment with the haute trend of hair chalki...,Beauty & Personal Care Hair Care Hair Colorin...,fashion angel color rox hair chox kit experi w...
4,Union Creative Giant Killing Figure 05: Daisu...,From Union Creative. Turn your display shelf i...,Toys & Games › Action Figures & Statues › Sta...,union creativ giant kill figur 05 : daisuk tsu...


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
tfidfv = TfidfVectorizer(tokenizer = tokenize_stem)

In [14]:
def cosine_sim(text1, text2):
  text1_concatenated = " ".join(text1)
  text2_concatenated = " ".join(text2)
  tfidf_matrix = tfidfv.fit_transform([text1_concatenated,text2_concatenated])
  return cosine_similarity(tfidf_matrix)[0][1]

In [15]:
def search_product(query):
  stemmed_query = tokenize_stem(query)
  amazon_df["similarity"] = amazon_df["stemmed_tokens"].apply(lambda x:cosine_sim(stemmed_query,x))
  res = amazon_df.sort_values(by = ["similarity"], ascending=False).head(10)[["Title","Description","Category"]]
  return res

In [16]:
amazon_df["Title"][20]

' Coronet Full Cheek Snaffle Pony Bit '

In [17]:
search_product(" Coronet Full Cheek Snaffle Pony Bit ")

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,Title,Description,Category
20,Coronet Full Cheek Snaffle Pony Bit,Coronet Full Cheek Pony Snaffle Bit The Full c...,Sports & Outdoors Sports & Fitness Other Spor...
304,Dodgeball - Peter La Fleur,Funko is a pop culture licensed-focused toy co...,Toys & Games Action Figures & Statues Action ...
243,"The Body Shop Drops of Youth Liquid Peel, 4.9...",The Drops of Youth™ Youth Liquid Peel exfoliat...,Automotive Motorcycle & Powersports Parts
57,NAO Love Always Porcelain Figurine,Nao porcelain sculptures are hand crafted in V...,Home & Kitchen Home Décor Home Décor Accents ...
551,"C2G 03836 2-Port Cat5e Surface Mount Box, Ivory","Ideal for surface mount applications, especial...",Electronics Computers & Accessories Computer ...
348,THE ONE Ankle Brace Black (EA),What you'll enjoy about the one ankle brace bl...,Health & Household › Medical Supplies & Equip...
415,Xit XT5CL 5-Piece Deluxe Cleaning Kit (White/...,The Xit 5 piece cleaning kit is the perfect ki...,Electronics › Camera & Photo › Accessories › ...
527,Cuteque International Victorian Lace Rose wit...,Cuteque International Inc. offered Victorian L...,Home & Kitchen Home Décor Artificial Plants &...
87,M.V. Trading Silicon Food Sushi Mold Cup for ...,"Creating muffins, cupcakes, brownies, or chees...",Home & Kitchen Kitchen & Dining Bakeware Past...
84,Body Fantasies Fresh White Musk 3 Piece Gift ...,Blends the clean fresh natural scents of brigh...,Beauty & Personal Care Fragrance Women's Sets
